In [92]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

File ‘bbc_text_cls.csv’ already there; not retrieving.



In [93]:
import numpy as np
import pandas as pd
import textwrap
import nltk
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [94]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [95]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [96]:
from nltk.corpus import wordnet
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [97]:
def get_wordnet_pos(treebank_tag):
  if treebank_tag.startswith('J'):
    return wordnet.ADJ
  elif treebank_tag.startswith('V'):
    return wordnet.VERB
  elif treebank_tag.startswith('N'):
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [98]:
df = pd.read_csv('bbc_text_cls.csv')

In [99]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [100]:
labels = set(df['labels'])
labels

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [101]:
# Pick a label whose data we want to train from
label = 'business'

In [102]:
texts = df[df['labels'] == label]['text']
texts.head()

0    Ad sales boost Time Warner profit\n\nQuarterly...
1    Dollar gains on Greenspan speech\n\nThe dollar...
2    Yukos unit buyer faces loan claim\n\nThe owner...
3    High fuel prices hit BA's profits\n\nBritish A...
4    Pernod takeover talk lifts Domecq\n\nShares in...
Name: text, dtype: object

In [103]:
get_wordnet_pos(nltk.pos_tag(['cat'])[0][1])

'n'

In [104]:
# collect counts
probs = {} # key: (w(t-1), w(t+1)), value: {w(t): count(w(t))}

for doc in texts:
  lines = doc.split("\n")
  for line in lines:
    tokens = word_tokenize(line)
    for i in range(len(tokens) - 2):
      t_0 = tokens[i]
      t_1 = tokens[i + 1]
      t_2 = tokens[i + 2]
      key = (t_0, t_2)
      if key not in probs:
        probs[key] = {}

      # add count for middle token
      pos = get_wordnet_pos(nltk.pos_tag([t_1])[0][1])
      if pos not in probs[key]:
        probs[key][pos] = {}

      if t_1 not in probs[key][pos]:
        probs[key][pos][t_1] = 1
      else:
        probs[key][pos][t_1] += 1

In [105]:
# normalize probabilities
for key, d in probs.items():
  # d should represent a distribution
  for keyPos,dPos in d.items():
    total = sum(dPos.values())
    for k, v in dPos.items():
      dPos[k] = v / total

In [106]:
probs

{('Ad', 'boost'): {'n': {'sales': 1.0}},
 ('sales', 'Time'): {'n': {'boost': 1.0}},
 ('boost', 'Warner'): {'n': {'Time': 1.0}},
 ('Time', 'profit'): {'n': {'Warner': 1.0}},
 ('Quarterly', 'at'): {'n': {'profits': 1.0}},
 ('profits', 'US'): {'n': {'at': 1.0}},
 ('at', 'media'): {'n': {'US': 1.0}},
 ('US',
  'giant'): {'n': {'media': 0.125,
   'telecoms': 0.125,
   'banking': 0.25,
   'foods': 0.125,
   'oil': 0.25,
   'mortgage': 0.125}, 'a': {'retail': 0.5, 'agrochemical': 0.5}},
 ('media', 'TimeWarner'): {'n': {'giant': 1.0}},
 ('giant', 'jumped'): {'n': {'TimeWarner': 1.0}},
 ('TimeWarner', '76'): {'n': {'jumped': 1.0}},
 ('jumped',
  '%'): {'n': {'76': 0.14285714285714285,
   '1.8': 0.14285714285714285,
   '11': 0.14285714285714285,
   '6': 0.14285714285714285,
   '10.7': 0.14285714285714285,
   '7': 0.14285714285714285,
   '22': 0.14285714285714285}},
 ('76', 'to'): {'n': {'%': 1.0}},
 ('%', '$'): {'n': {'to': 0.7727272727272727, 'at': 0.22727272727272727}},
 ('to', '1.13bn'): {'n'

In [107]:
texts.iloc[0].split("\n")

['Ad sales boost Time Warner profit',
 '',
 'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.',
 '',
 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.',
 '',
 "Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers a

In [108]:
def spin_document(doc):
  # split the document into lines (paragraphs)
  lines = doc.split("\n")
  output = []
  for line in lines:
    if line:
      new_line = spin_line(line)
    else:
      new_line = line
    output.append(new_line)
  return "\n".join(output)

In [109]:
detokenizer = TreebankWordDetokenizer()

In [110]:
texts.iloc[0].split("\n")[2]

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.'

In [111]:
detokenizer.detokenize(word_tokenize(texts.iloc[0].split("\n")[2]))

'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.'

In [112]:
def sample_word(d):
  p0 = np.random.random()
  cumulative = 0
  for t, p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t
  assert(False) # should never get here

In [113]:
def spin_line(line):
  tokens = word_tokenize(line)
  i = 0
  output = [tokens[0]]
  while i < (len(tokens) - 2):
    t_0 = tokens[i]
    t_1 = tokens[i + 1]
    t_2 = tokens[i + 2]
    key = (t_0, t_2)
    pos = get_wordnet_pos(nltk.pos_tag([t_1])[0][1])
    p_dist = probs[key][pos]
    if len(p_dist) > 1 and np.random.random() < 0.3:
      # let's replace the middle word
      middle = sample_word(p_dist)
      output.append(t_1)
      output.append("<" + middle + ">")
      output.append(t_2)

      # we won't replace the 3rd token since the middle
      # token was dependent on it
      # instead, skip ahead 2 steps
      i += 2
    else:
      # we won't replace this middle word
      output.append(t_1)
      i += 1
  # append the final token - only if there was no replacement
  if i == len(tokens) - 2:
    output.append(tokens[-1])
  return detokenizer.detokenize(output)

In [114]:
np.random.seed(1234)

In [115]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]
new_doc = spin_document(doc)

In [116]:
print(textwrap.fill(
    new_doc, replace_whitespace=False, fix_sentence_endings=True))

Bombardier chief to leave company

Shares in train and plane-making
giant Bombardier have fallen to a 10-year <five-year> low following
the departure of its <Japan> chief executive and two members <thirds>
of the board.

Paul Tellier <Sheard>, who was <was> also Bombardier's
president <scandal>, left the company amid an ongoing restructuring .
Laurent Beaudoin, part <part> of the family that controls the
Montreal-based firm, will take on the role of CEO <2007> under a newly
created management structure . Analysts said the resignations seem to
have stemmed from a boardroom dispute . Under <While> Mr Tellier's
tenure at the company, which began in January <January> 2003, plans to
cut the worldwide workforce of 75,000 <WMC> by almost <nearly> a third
by 2006 were announced . The firm's snowmobile <snowmobile> division
and defence <ground> services unit were also sold and Bombardier
started the development of a <the> new aircraft seating 110 to 135
passengers.

Mr Tellier had indicated he 